# Daily Pricer — Corners, Goals, Parlay (one-side-only; bankroll-capped)

In [ ]:

from pathlib import Path
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import corners_edge as ce
importlib.reload(ce)

BASE = Path.cwd()
ARTIFACT_PREFIX = BASE / "corners_model"
INPUT_CSV  = BASE / "data" / "matchday_inputs.csv"
OUTPUT_CSV = BASE / "data" / "matchday_quotes.csv"

BANKROLL = 50.0
KELLY_FRACTION = 0.5

MIN_EV = 0.0
MIN_EDGE_PCT = 0.0
MIN_KELLY = 0.0
MIN_STAKE_DOLLARS = 0.0

draws, team_names, div_names, zstats = ce.load_artifacts(ARTIFACT_PREFIX)
print("Artifacts loaded.", len(team_names), "teams")


In [ ]:

# Read CSV, clean odds (auto-detect tab/comma)
inp = pd.read_csv(INPUT_CSV, sep=None, engine="python", dtype=str).fillna("")
inp = inp.applymap(lambda x: x.strip() if isinstance(x, str) else x)
MISSING = {"", "na", "n/a", "nan", "-", "--"}

def to_float(x):
    try: return float(x)
    except Exception: return np.nan

for c in ["book_over","book_under","book_exact","odds_home","odds_draw","odds_away","odds_over25","odds_under25","goals_over","goals_under","parlay_price"]:
    if c in inp.columns:
        inp[c] = inp[c].apply(lambda s: "" if str(s).lower() in MISSING else s)
        inp[c] = inp[c].apply(ce.parse_odds_decimal)

inp["corners_line"] = inp.get("corners_line", np.nan).apply(to_float)
inp["exact_total"] = inp.get("exact_total", np.nan).apply(to_float)
inp["goals_line"] = inp.get("goals_line", 2.5).apply(lambda x: float(x) if pd.notna(x) else 2.5)
inp["rho_copula"] = inp.get("rho_copula", 0.2).apply(lambda x: float(x) if str(x).lower() not in MISSING else 0.2)

display(inp.head())


In [ ]:

def _f(x, default=None):
    try: return float(x)
    except Exception: return default

def normalize_feature_odds(row, zstats):
    oh = row.get("odds_home"); od = row.get("odds_draw"); oa = row.get("odds_away")
    o25 = row.get("odds_over25"); u25 = row.get("odds_under25")
    if any(v in (None, np.nan) for v in [oh, od, oa]):
        p_home = float(zstats.mean_p_home); p_draw = (1.0 - p_home)/2.0; p_away = p_draw
        oh, od, oa = 1.0/p_home, 1.0/p_draw, 1.0/p_away
    if any(v in (None, np.nan) for v in [o25, u25]):
        p_over = float(zstats.mean_p_over25); p_under = 1.0 - p_over
        o25, u25 = 1.0/p_over, 1.0/p_under
    return oh, od, oa, o25, u25

def gating(ev, kelly, edge_pct, price):
    ok = True; reasons = []
    if not (np.isfinite(ev) and ev > MIN_EV): ok=False; reasons.append("EV<th")
    if not (kelly > MIN_KELLY): ok=False; reasons.append("Kelly<th")
    if not (np.isfinite(edge_pct) and edge_pct >= MIN_EDGE_PCT): ok=False; reasons.append("Edge<th")
    if not (price and price > 1.0): ok=False; reasons.append("Price")
    return ok, ",".join(reasons) if not ok else None


In [ ]:

rows = []
for _, r in inp.iterrows():
    home = str(r.get("home_team","")).strip()
    away = str(r.get("away_team","")).strip()
    div  = str(r.get("division","")).strip() or None
    line = _f(r.get("corners_line"), 9.5)

    price_over  = r.get("book_over")
    price_under = r.get("book_under")
    price_exact = r.get("book_exact")
    exact_total = _f(r.get("exact_total"))

    oh, od, oa, o25, u25 = normalize_feature_odds(r, zstats)

    # Quick expected corners
    res = ce.predict_game(draws, team_names, div_names, zstats,
                          home, away, div, oh, od, oa, o25, u25, line,
                          price_over, price_under, n_sims=15000)
    expected = float(res["expected_corners"])

    # Full PMF (for push-aware probs)
    def idx(name, universe):
        try: return universe.index(name)
        except ValueError: return 0
    hi = idx(home, team_names); ai = idx(away, team_names)
    if isinstance(div, str) and len(div_names)>0:
        try: di = div_names.index(div)
        except ValueError: di = 0
    else:
        di = 0
    intercept = draws["intercept"].reshape(-1)
    home_adv  = draws["home_adv"].reshape(-1)
    team_home_eff = draws["team_home_eff"]
    team_away_eff = draws["team_away_eff"]
    alpha = draws["alpha"].reshape(-1)
    beta_homeprob = draws.get("beta_homeprob", np.zeros_like(intercept)).reshape(-1)
    beta_over25   = draws.get("beta_over25",   np.zeros_like(intercept)).reshape(-1)
    div_term = draws["div_eff"][:, di].reshape(-1) if ("div_eff" in draws and len(div_names)>0) else 0.0
    p_home, p_draw, p_away = ce.implied_probs_1x2_from_odds(oh, od, oa)
    p_over25 = ce.implied_prob_over25_from_odds(o25, u25)
    z_ph = float(np.clip(ce.zscore(p_home,  zstats.mean_p_home,  zstats.std_p_home),  -6.0, 6.0))
    z_po = float(np.clip(ce.zscore(p_over25, zstats.mean_p_over25, zstats.std_p_over25), -6.0, 6.0))
    eta = (intercept + home_adv + team_home_eff[:, hi] + team_away_eff[:, ai]
           + div_term + beta_homeprob * z_ph + beta_over25 * z_po)
    mu = np.exp(eta); mu = np.clip(mu, 1e-6, 60.0)

    sims = ce.nb_predictive_samples(mu, alpha, n_sims=40000)
    pmf = ce.pmf_from_draws(sims, vmax=int(np.clip(np.percentile(sims, 99.5), 20, 60)))
    p_over, p_under, p_push = ce.prob_over_under_push(pmf, line)

    ev_over,  k_over  = ce.ev_kelly_two_way(p_over,  p_under, price_over,  KELLY_FRACTION)
    ev_under, k_under = ce.ev_kelly_two_way(p_under, p_over,  price_under, KELLY_FRACTION)
    be_over  = (1 - p_push) / price_over  if price_over  and price_over  > 1.0 else np.nan
    be_under = (1 - p_push) / price_under if price_under and price_under > 1.0 else np.nan
    edge_over  = (p_over  - be_over)  * 100 if np.isfinite(be_over)  else np.nan
    edge_under = (p_under - be_under) * 100 if np.isfinite(be_under) else np.nan

    corners_best_side  = "over" if (ev_over if np.isfinite(ev_over) else -np.inf) >= (ev_under if np.isfinite(ev_under) else -np.inf) else "under"
    corners_best_ev    = ev_over if corners_best_side=="over" else ev_under
    corners_best_kelly = k_over  if corners_best_side=="over" else k_under
    corners_best_edge  = edge_over if corners_best_side=="over" else edge_under
    corners_best_price = price_over if corners_best_side=="over" else price_under
    ok_corners, reasons_corners = gating(corners_best_ev, corners_best_kelly, corners_best_edge, corners_best_price)
    stake_over = stake_under = 0.0
    if ok_corners:
        if corners_best_side == "over": stake_over = BANKROLL * corners_best_kelly
        else: stake_under = BANKROLL * corners_best_kelly

    fair_exact = None; ev_exact = None; kelly_exact = None; p_exact = None
    if not np.isnan(exact_total):
        p_exact = pmf.get(int(round(exact_total)), 0.0)
        fair_exact = (1.0 / p_exact) if (p_exact and p_exact > 0) else None
        if price_exact and fair_exact:
            ev_exact, kelly_exact = ce.ev_kelly_single(p_exact, price_exact, KELLY_FRACTION)

    # Goals
    g_line = r.get("goals_line", 2.5)
    g_over_price  = r.get("goals_over", None)
    g_under_price = r.get("goals_under", None)
    p_goals_over = None
    ev_goals_over = ev_goals_under = np.nan
    k_goals_over = k_goals_under = 0.0
    be_goals_over = be_goals_under = np.nan
    edge_goals_over = edge_goals_under = np.nan

    if g_over_price and g_under_price:
        p_goals_over = ce.implied_prob_two_way(g_over_price, g_under_price)
        p_goals_under = 1.0 - p_goals_over
        be_goals_over  = 1.0 / g_over_price
        be_goals_under = 1.0 / g_under_price
        edge_goals_over  = (p_goals_over  - be_goals_over)  * 100.0
        edge_goals_under = (p_goals_under - be_goals_under) * 100.0
        ev_goals_over,  k_goals_over  = ce.ev_kelly_two_way(p_goals_over,  p_goals_under, g_over_price,  KELLY_FRACTION)
        ev_goals_under, k_goals_under = ce.ev_kelly_two_way(p_goals_under, p_goals_over,  g_under_price, KELLY_FRACTION)

    goals_take_side = "no_bet"
    goals_stake_over = goals_stake_under = 0.0
    if g_over_price and g_under_price:
        goals_best_side  = "over" if (ev_goals_over if np.isfinite(ev_goals_over) else -np.inf) >= (ev_goals_under if np.isfinite(ev_goals_under) else -np.inf) else "under"
        goals_best_ev    = ev_goals_over if goals_best_side=="over" else ev_goals_under
        goals_best_kelly = k_goals_over  if goals_best_side=="over" else k_goals_under
        goals_best_edge  = edge_goals_over if goals_best_side=="over" else edge_goals_under
        goals_best_price = g_over_price if goals_best_side=="over" else g_under_price
        ok_goals, reasons_goals = gating(goals_best_ev, goals_best_kelly, goals_best_edge, goals_best_price)
        if ok_goals:
            goals_take_side = goals_best_side
            if goals_best_side == "over": goals_stake_over = BANKROLL * goals_best_kelly
            else: goals_stake_under = BANKROLL * goals_best_kelly
        else:
            reasons_goals = reasons_goals
    else:
        reasons_goals = "no_prices"

    # Parlay (only if both singles EV>0 and parlay EV>0)
    parlay_joint = None
    parlay_price_used = None
    parlay_ev = np.nan; parlay_k = 0.0; parlay_stake = 0.0
    rho = float(r.get("rho_copula", 0.2))
    parlay_price = r.get("parlay_price", None)

    corners_side = ("over" if stake_over>stake_under else ("under" if stake_under>stake_over else None))
    goals_side   = ("over" if goals_stake_over>goals_stake_under else ("under" if goals_stake_under>goals_stake_over else None))

    if (corners_side is not None) and (goals_side is not None) and (p_goals_over is not None):
        pA = p_over if corners_side=="over" else p_under
        pB = p_goals_over if goals_side=="over" else (1.0 - p_goals_over)
        parlay_joint = ce.gaussian_copula_joint(pA, pB, rho=rho, n=200_000, seed=123)
        price_leg1 = price_over if corners_side=="over" else price_under
        price_leg2 = g_over_price if goals_side=="over" else g_under_price
        parlay_price_used = parlay_price or (price_leg1 * price_leg2 if price_leg1 and price_leg2 else None)
        if parlay_price_used:
            parlay_ev, parlay_k = ce.ev_kelly_single(parlay_joint, parlay_price_used, KELLY_FRACTION)
            ev_single1 = ev_over if corners_side=="over" else ev_under
            ev_single2 = ev_goals_over if goals_side=="over" else ev_goals_under
            if (np.isfinite(ev_single1) and ev_single1 > 0) and (np.isfinite(ev_single2) and ev_single2 > 0) and (np.isfinite(parlay_ev) and parlay_ev > 0):
                parlay_stake = BANKROLL * max(0.0, parlay_k)

    rows.append({
        "home_team": home, "away_team": away, "division": div,
        "corners_line": float(line) if line is not None else None,
        "expected_corners": round(expected, 3),
        "p_over": round(p_over, 4), "p_under": round(p_under, 4), "p_push": round(p_push, 4),
        "book_over": price_over, "book_under": price_under,
        "EV_over_per_$": round(ev_over, 4) if np.isfinite(ev_over) else None,
        "EV_under_per_$": round(ev_under, 4) if np.isfinite(ev_under) else None,
        "stake_over": round(stake_over, 2), "stake_under": round(stake_under, 2),
        "take_side": ("over" if stake_over>stake_under else ("under" if stake_under>stake_over else "no_bet")),
        "goals_line": g_line,
        "book_goals_over": g_over_price, "book_goals_under": g_under_price,
        "p_goals_over": (round(p_goals_over,4) if p_goals_over is not None else None),
        "EV_goals_over_per_$": (round(ev_goals_over,4) if np.isfinite(ev_goals_over) else None),
        "EV_goals_under_per_$": (round(ev_goals_under,4) if np.isfinite(ev_goals_under) else None),
        "goals_stake_over": round(goals_stake_over, 2), "goals_stake_under": round(goals_stake_under, 2),
        "goals_take_side": goals_take_side,
        "rho_copula": rho,
        "parlay_joint_prob": (round(parlay_joint,4) if parlay_joint is not None else None),
        "parlay_price_used": parlay_price_used,
        "parlay_EV_per_$": (round(parlay_ev,4) if np.isfinite(parlay_ev) else None),
        "parlay_kelly": (round(parlay_k,4) if np.isfinite(parlay_k) else None),
        "parlay_stake": round(parlay_stake, 2),
        "exact_total": exact_total, "book_exact": price_exact,
        "p_exact": (round(p_exact,4) if p_exact is not None else None),
        "fair_exact": (round(fair_exact,3) if fair_exact is not None else None),
        "EV_exact_per_$": (round(ev_exact,4) if ev_exact is not None else None),
        "kelly_exact": (round(kelly_exact,4) if kelly_exact is not None else None),
    })

quotes = pd.DataFrame(rows)
display(quotes)


In [ ]:

# Bankroll cap across all bets on the slate (sum ≤ BANKROLL)
quotes["raw_stake"] = 0.0
quotes["raw_stake"] += quotes[["stake_over","stake_under"]].max(axis=1).fillna(0.0)
quotes["raw_stake"] += quotes[["goals_stake_over","goals_stake_under"]].max(axis=1).fillna(0.0)
quotes["raw_stake"] += quotes["parlay_stake"].fillna(0.0)

total_raw = float(quotes["raw_stake"].sum())
if total_raw > 0:
    target_total = BANKROLL * 1.0
    scale = min(1.0, target_total / total_raw)
    alloc = (quotes["raw_stake"] * scale).astype(float)
    alloc_rounded = alloc.round(2)
    diff = round(target_total - alloc_rounded.sum(), 2)
    if abs(diff) >= 0.01:
        idx_max = alloc_rounded.idxmax()
        alloc_rounded.loc[idx_max] = max(0.0, alloc_rounded.loc[idx_max] + diff)
    quotes["stake_final_total"] = alloc_rounded
else:
    quotes["stake_final_total"] = 0.0

print(f"Total final stake across corners+goals+parlays: {quotes['stake_final_total'].sum():.2f} / {BANKROLL:.2f}")
display(quotes[["home_team","away_team","division","stake_final_total"]])


In [ ]:

# Save output
OUTPUT_CSV.parent.mkdir(parents=True, exist_ok=True)
quotes.to_csv(OUTPUT_CSV, index=False)
print("Saved:", OUTPUT_CSV)
